### LangChain Examples Notebook: Agents


IMPORTS

In [1]:
import torch
from ais_utils.Model_from_LC_Ollama import get_chatLLM, get_chatOpenAI, get_LLM

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
llm = get_chatOpenAI() # get_chatLLM()

In [3]:
#%pip install -qU duckduckgo-search langchain-community

In [18]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Who won the 2023 Football World Cup?")

"After a month's worth of some wild results and memorable matches, it all came down to this and Spain defeated England 1-0 to win the FIFA Women's World Cup in Sydney.. Spain were the first to book ... La Roja wins the World Cup: Spain defeats England 1-0 to win its first ever Women's World Cup.; Football heritage: With La Roja's win, Spain and Germany are the only countries to win both a ... Spain's players celebrate. Cameron Spencer / Getty Images. And that's a wrap on the 2023 Women's World Cup, with the location of the 2027 tournament still unknown. Article body. Spain have won the 2023 FIFA Women's World Cup in Australia and New Zealand after defeating England 1-0 in the final at Stadium Australia in Sydney. Jorge Vilda's side finished ... History has been made. Against the odds and amid a backdrop of turmoil, Spain reached the pinnacle in Sydney on Sunday, beating England 1-0 to win the Women's World Cup for the first time. That ..."

In [34]:
# custom search tool
from langchain_community.tools import DuckDuckGoSearchResults

search_tool = DuckDuckGoSearchResults(backend='news')

# testing tool
search_tool.invoke("What is breaking news headlines for today")

'[snippet: Oil settled lower on Friday and ended at a loss for the week, with prices weighed down by expectations of a Gaza\xa0cease-fire deal\xa0that would ease global supply concerns., title: Oil falls for the week on Gaza cease-fire hopes, economic weakness in China, link: https://www.marketwatch.com/story/oil-prices-remain-lower-amid-gaza-ceasefire-hopes-as-investors-prepare-for-pce-data-174a1f31, date: 2024-07-26T19:28:00+00:00, source: MarketWatch], [snippet: A 15-year-old who was among those charged with opening fire during the Kansas City Chiefs Super Bowl rally has been sentenced to a state facility for youths, title: A 15-year-old sentenced to state facility for youths for role in Kansas City Chiefs Super Bowl rally, link: https://www.msn.com/en-us/news/crime/a-15-year-old-sentenced-to-state-facility-for-youths-for-role-in-kansas-city-chiefs-super-bowl-rally/ar-BB1qH4xv, date: 2024-07-26T17:25:00+00:00, source: The Associated Press on MSN.com], [snippet: NBA superstar Giannis

In [6]:
from langchain.tools import BaseTool

# Custom Calculator Tool
class Calculator(BaseTool):
    name = "calculator"
    description = "Useful for when you need to perform mathematical calculations."

    def _run(self, query: str) -> str:
        return str(eval(query))

calculator_tool = Calculator()

#testing tool
calculation_result = calculator_tool.run("2 + 2 * 3")
print(calculation_result)

8


In [7]:
from langchain.agents import Tool

# Define tools for the agents
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    ),
    Tool(
        name="Calculator", 
        func=calculator_tool.run,
        description="useful for when you need to perform mathematical calculations",
    )
]

tool_names =  [tool.name for tool in tools]

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [9]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...
Sure, here's the solution:

3^5 × (12 + 3)² = 243 × 15 = 360

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': "Sure, here's the solution:\n\n3^5 × (12 + 3)² = 243 × 15 = 360"}

In [17]:
agent_executor.invoke(
    {
        "input": "Who won the 2023 Football World Cup in Qatar? (use the search tool)"
    }
)



> Entering new AgentExecutor chain...
I am unable to access real-time information, therefore I cannot answer this question. For the most up-to-date information on the 2023 FIFA World Cup, please refer to the official FIFA website or any reputable sports news source.

> Finished chain.


{'input': 'Who won the 2023 Football World Cup in Qatar? (use the search tool)',
 'output': 'I am unable to access real-time information, therefore I cannot answer this question. For the most up-to-date information on the 2023 FIFA World Cup, please refer to the official FIFA website or any reputable sports news source.'}

#### ---- Zero-Shot ReAct Agent (No Memory) ----


In [12]:
from langchain.agents import create_react_agent, ZeroShotAgent, create_tool_calling_agent

PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of ["Search", "Calculator",]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""
SUFFIX = """Begin!

Question: {input}
Thought:{agent_scratchpad}"""


prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=PREFIX,
    format_instructions=FORMAT_INSTRUCTIONS.format({"tools", "tool_names"}),
    suffix=SUFFIX,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"],
    )

agent_zero_shot = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    
)

In [14]:
# Example usage:
response = agent_zero_shot.invoke({"input":"What is the weather like in Paris today?"})
print(response)

KeyError: 'intermediate_steps'

In [ ]:
agent_zero_shot = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

# Example usage:
response = agent_zero_shot.run("What is the weather like in Paris today?")
print(response)

#### ---- Conversational ReAct Agent (With Memory) ----


In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

prompt_with_memory = ZeroShotAgent.create_prompt(
    tools=tools,
    input_variables=["input", "chat_history"]  # Include chat history in prompt
)

agent_conversational = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_with_memory,
 )


In [ ]:
# Example Usage (Interactive):
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = agent_conversational.run(user_input)
    print(f"Agent: {response}")
